# TSIS 11 - Phonebook

Это **#11** лабораторная работа по предмету *"Принципы Программирования 2".*

Студента: **Айбасов Даниал Елнарович**

In [28]:
import psycopg2

conn = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="123",
    host="localhost",
    port="1337"
)

cur = conn.cursor()

### 1. Function that returns all records based on a pattern (example of pattern: part of name, surname, phone number)

In [2]:
pattern = """
CREATE OR REPLACE FUNCTION get_data(pattern VARCHAR)
RETURNS SETOF PhoneBook AS $$
BEGIN
    RETURN QUERY SELECT * FROM PhoneBook WHERE first_name LIKE '%' || pattern || '%' OR last_name LIKE '%' || pattern || '%' OR phone LIKE '%' || pattern || '%';
END;
$$ LANGUAGE plpgsql;
"""

In [17]:
try:
    cur.execute(pattern)
    cur.callproc('get_data', ('A',)) 
 
    result = cur.fetchall()
    print( result )
except psycopg2.DatabaseError as e:
    print(f"Error: {e}")
    conn.rollback()
finally:
    conn.commit()

[(1, 'Danial', 'Aibassov', '+77777')]


### 2. Create procedure to insert new user by name and phone, update phone if user already exists

In [3]:
procedure = """
CREATE OR REPLACE PROCEDURE insert_user(IN name VARCHAR, IN surname VARCHAR, IN phone VARCHAR)
LANGUAGE plpgsql
AS $$
BEGIN
    IF EXISTS(SELECT 1 FROM PhoneBook WHERE first_name = insert_user.name AND last_name = insert_user.surname) THEN
        UPDATE PhoneBook SET phone = insert_user.phone WHERE first_name = insert_user.name AND last_name = insert_user.surname;
    ELSE
        INSERT INTO PhoneBook (first_name, last_name, phone) VALUES (insert_user.name, insert_user.surname, insert_user.phone);
    END IF;
END;
$$;
"""


In [18]:
try:
    cur.execute(procedure)
    cur.execute("CALL insert_user(%s, %s, %s)", ['Aziz', 'Doe', '871400'])
    result = cur.fetchall()
    print( result )
except psycopg2.DatabaseError as e:
    print(f"Error: {e}")
    conn.rollback()
finally:
    conn.commit()


Error: no results to fetch


### 3. Create procedure to insert many new users by list of name and phone. Use loop and if statement in stored procedure. Check correctness of phone in procedure and return all incorrect data.

In [29]:
inserting_names = """
CREATE OR REPLACE PROCEDURE insert_users(
    IN user_list text[][]
)
LANGUAGE plpgsql
AS $$
DECLARE
    i INTEGER := 1;
    current_user text[];
BEGIN
    WHILE i <= array_length(user_list, 1) LOOP
        current_user := user_list[i];
        
        IF regexp_matches(current_user.phone, '[0-9]+') THEN
            IF EXISTS(SELECT 1 FROM PhoneBook WHERE first_name = current_user.first_name AND last_name = current_user.last_name) THEN
                UPDATE PhoneBook SET phone = current_user.phone WHERE first_name = current_user.first_name AND last_name = current_user.last_name;
            ELSE
                INSERT INTO PhoneBook (first_name, last_name, phone) VALUES (current_user.first_name, current_user.last_name, current_user.phone);
            END IF;
        END IF;
        
        i := i + 1;
    END LOOP;
END;
$$;
"""


In [30]:
try:
    cur.execute( inserting_names )
    cur.execute("CALL insert_users(%s, %s, %s)", [
        ('John', 'Doe', '555-1234'),
        ('Jane', 'Doe', 'gfg'),
        ('Bob', 'Smith', '1234')
    ])
    result = cur.fetchall()
    print( result )
except psycopg2.DatabaseError as e:
    print(f"Error: {e}")
    conn.rollback()
finally:
    conn.commit()

Error: ОШИБКА:  ошибка синтаксиса (примерное положение: "current_user")
LINE 12:         current_user := user_list[i];
                 ^



### 4. Create function to querying data from the tables with pagination (by limit and offset)

In [51]:
queue = """
CREATE OR REPLACE FUNCTION get_data1(
    table_name text, 
    limit_rows integer, 
    offset_rows integer
) RETURNS SETOF anyelement AS $$
BEGIN
    RETURN QUERY EXECUTE format('SELECT * FROM %I LIMIT %s OFFSET %s', table_name, limit, offset);
END;
$$ LANGUAGE plpgsql;
"""

In [55]:
queue = """
CREATE OR REPLACE FUNCTION query_phonebook(
    table_name TEXT, 
    limit_rows INTEGER, 
    offset_rows INTEGER
) RETURNS SETOF RECORD AS $$
BEGIN
    RETURN QUERY EXECUTE format('SELECT * FROM %I LIMIT %s OFFSET %s', table_name, limit_rows, offset_rows);
END;
$$ LANGUAGE plpgsql;
"""

In [44]:
queue = """
CREATE OR REPLACE FUNCTION query_phonebook(
    limit_rows INT,
    offset_rows INT
)
RETURNS TABLE(id INT, first_name TEXT, last_name TEXT, phone TEXT)
LANGUAGE plpgsql
AS $$
BEGIN
    RETURN QUERY
    SELECT id, first_name, last_name, phone
    FROM Phonebook
    ORDER BY id
    LIMIT limit_rows OFFSET offset_rows;
END;
$$;
"""

In [56]:
try:
    cur.execute( queue )
    cur.callproc( 'query_phonebook', ('PhoneBook', 2, 1))
    result = cur.fetchall()
    print( result )
except psycopg2.DatabaseError as e:
    print(f"Error: {e}")
    conn.rollback()
finally:
    conn.commit()

Error: ОШИБКА:  у функций, возвращающих запись, должен быть список определений столбцов
LINE 1: SELECT * FROM query_phonebook('PhoneBook',2,1)
                      ^



### 5. Implement procedure to deleting data from tables by username or phone

In [6]:
delete = """
CREATE OR REPLACE PROCEDURE delete_data(IN data VARCHAR(30))
LANGUAGE plpgsql
AS $$
BEGIN
    DELETE FROM PhoneBook WHERE first_name = data OR last_name = data OR phone = data;
END;
$$;
"""

In [ ]:
try:
    # cur.execute(pattern)
    # cur.callproc('get_data', ('A',)) 

    # cur.execute(procedure)
    # cur.execute("CALL insert_user(%s, %s, %s)", ['Aziz', 'Doe', '871400'])

    # cur.execute( inserting_names )
    # cur.execute("CALL insert_users(%s, %s, %s)", [
    #     ('John', 'Doe', '555-1234'),
    #     ('Jane', 'Doe', 'gfg'),
    #     ('Bob', 'Smith', '1234')
    # ])

    # cur.execute( queue )
    # cur.callproc( 'get_data1', ('PhoneBook', 2, 1))

    cur.execute( delete )
    cur.execute('CALL delete_data(%s)', ('Doe',))
    
    # result = cur.fetchall()
    # print( result )
except psycopg2.DatabaseError as e:
    print(f"Error: {e}")
    conn.rollback()
finally:
    conn.commit()
    cur.close()
    conn.close()